In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [53]:
# loading businesses and restaurant ids
df_bus = pd.read_csv("/Users/dima/code/Dimasaur/scorecast/data/restaurants_limited_features.csv")
res_ids = pd.read_csv("/Users/dima/code/Dimasaur/scorecast/data/restaurants.csv")

/var/folders/fd/585jf60x39vflmpy4gfz7yzh0000gn/T/ipykernel_43335/1280512225.py:2: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bus = pd.read_csv("/Users/dima/code/Dimasaur/scorecast/data/restaurants_limited_features.csv")


In [54]:
df_bus['food_type_string'] = df_bus['food_type'].apply(lambda x: str(x).strip('[]'))
df_bus['food_type_string'] = df_bus['food_type_string'].apply(lambda x: str(x).replace("'",""))

In [55]:
df_bus.food_type_string.value_counts(normalize=True).head(20)

food_type_string
O                         0.624570
Pizza                     0.033768
Mexican                   0.027416
Chinese                   0.026020
Burgers, Fast Food        0.015937
Italian, Pizza            0.012972
Italian                   0.011843
American (New)            0.010293
American (Traditional)    0.009700
Fast Food, Sandwiches     0.009700
Japanese, Sushi Bars      0.006275
                          0.006046
Thai                      0.005606
Sandwiches                0.005491
Barbeque                  0.005453
Vietnamese                0.005204
Seafood                   0.005108
Diners                    0.005089
Coffee & Tea, Food        0.004458
Indian                    0.004420
Name: proportion, dtype: float64

In [56]:
replace_dictionary = {
    "Pizza" : "Italian",
    "Burgers, Fast Food" : "Burgers",
    "Italian, Pizza" : "Italian",
    "American (New)" : "American",
    "Fast Food, Sandwiches" : "Sandwiches",
    "American (Traditional)" : "American",
    "Fast Food, Mexican" : "Mexican",
    "Coffee & Tea, Food" : "Cafes",
    "Japanese, Sushi Bars" : "Japanese",
    "American (Traditional), Diners" : "Diners",
    "Bakeries, Food" : "Bakery",
    "Delis, Sandwiches" : "Delis",
    "Mexican, Tex-Mex" : "Mexican",
    "American (Traditional), Burgers" : "American",
    "Sushi Bars" : "Japanese",
    "Chinese, Fast Food" : "Chinese",
    "American (Traditional), Cafes" : "American",
    "Mexican, Tacos" : "Mexican",
    "Pizza, Sandwiches" : "Sandwiches",
    "Food, Pizza" : "Italian",
    "American (Traditional), Seafood" : "American",
    "Greek, Mediterranean" : "Greek",
    "American (New), American (Traditional)" : "American",
    "Cajun/Creole, Seafood" : "Cajun/Creole"

}

In [57]:
df_bus.food_type_string = df_bus.food_type_string.replace(replace_dictionary)

In [89]:
df_food_t20 = list(df_bus.food_type_string.value_counts(normalize=True).head(25).index)
df_food_t20.remove('O')
df_food_t20.remove('')
df_food_t20 = df_food_t20[:20]

In [90]:
df_food_t20

['Italian',
 'Mexican',
 'Chinese',
 'American',
 'Burgers',
 'Sandwiches',
 'Japanese',
 'Diners',
 'Cafes',
 'Delis',
 'Thai',
 'Barbeque',
 'Vietnamese',
 'Seafood',
 'Indian',
 'Greek',
 'Cajun/Creole',
 'Bakery',
 'Steakhouses',
 'Chicken Wings']

In [94]:
np_food_20 = np.array([df_food_t20])

In [95]:
ohe = OneHotEncoder(sparse_output=False)
ohe.fit(np_food_20)

OneHotEncoder(sparse_output=False)

In [96]:
# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe.categories_}")

The categories detected by the OneHotEncoder are [array(['Italian'], dtype='<U13'), array(['Mexican'], dtype='<U13'), array(['Chinese'], dtype='<U13'), array(['American'], dtype='<U13'), array(['Burgers'], dtype='<U13'), array(['Sandwiches'], dtype='<U13'), array(['Japanese'], dtype='<U13'), array(['Diners'], dtype='<U13'), array(['Cafes'], dtype='<U13'), array(['Delis'], dtype='<U13'), array(['Thai'], dtype='<U13'), array(['Barbeque'], dtype='<U13'), array(['Vietnamese'], dtype='<U13'), array(['Seafood'], dtype='<U13'), array(['Indian'], dtype='<U13'), array(['Greek'], dtype='<U13'), array(['Cajun/Creole'], dtype='<U13'), array(['Bakery'], dtype='<U13'), array(['Steakhouses'], dtype='<U13'), array(['Chicken Wings'], dtype='<U13')]
